In [3]:
import re
from tqdm import tqdm # progress bar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import pycld2 as cld2  # language detector
from unidecode import unidecode
import tldextract # domain extractor

In [211]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', None)

## load the dataset

In [4]:
wiki = load_dataset("wikimedia/wikipedia", "20231101.az")["train"].to_pandas()
cultrax_df = load_dataset("uonlp/CulturaX", "az")['train'].to_pandas()
print(wiki.shape)
print(cultrax_df.shape)
cultrax_df['source'].value_counts()

Loading dataset shards:   0%|          | 0/33 [00:00<?, ?it/s]

(196158, 4)
(5084505, 4)


source
mC4           4306983
OSCAR-2019     290336
OSCAR-2109     212313
OSCAR-2301     177332
OSCAR-2201      97541
Name: count, dtype: int64

## concat cultrax and wikipedia

In [5]:
# drop wikipedia data to add from the wikipedia itself
cultrax_df = cultrax_df[~cultrax_df['url'].str.contains("wikipedia")]
cultrax_df.drop(columns=['timestamp'], axis=1, inplace=True)
wiki.drop(columns=['id', 'title'], axis=1, inplace=True)
wiki['source'] = 'wiki'
cultrax = pd.concat(objs=[cultrax_df, wiki], axis=0)
print(cultrax.shape)

(5208062, 3)


In [6]:
cultrax['source'].value_counts()

source
mC4           4254852
OSCAR-2019     290336
OSCAR-2109     199080
wiki           196158
OSCAR-2301     172705
OSCAR-2201      94931
Name: count, dtype: int64

## extract the domains from the urls

In [7]:
tqdm.pandas(desc="Processing URLs")
cultrax['domain'] = cultrax['url'].progress_apply(lambda x: tldextract.extract(x).domain)

Processing URLs: 100%|██████████| 5208062/5208062 [00:15<00:00, 340445.35it/s]


In [8]:
cultrax['domain'].value_counts()

domain
                      290336
wikipedia             196158
azadliq               179510
trend                  99718
report                 67497
                       ...  
documentfoundation         1
skygolf                    1
hanuur                     1
brasilalemanha             1
linkthe                    1
Name: count, Length: 29123, dtype: int64

In [9]:
print(cultrax[cultrax['domain'] == ""].iloc[1000]['text'])

DİN-in mətbuat xidmətindən "Report"a verilən məlumata görə, mayın 2-də saat 19 radələrində Goranboy rayonunun Xanqərvənd kəndi ərazisində kənd sakini Əli Süleymanovun idarə etdiyi “VAZ-2121” və Ağdam rayon sakini Füzuli Quliyevin idarə etdiyi “Toyota” markalı avtomobillər toqquşub. Nəticədə Ə.Süleymanov hadisə yerində ölüb, 2 sərnişini, habelə F.Quliyev və sərnişinləri - arvadı Cəmilə və azyaşlı nəvəsi xəstəxanaya yerləşdiriliblər. F.Quliyev və Cəmilə orada ölüblər.


## language detection

In [10]:
def detect_language(text):
    try:
        det = cld2.detect(text)
        is_reliable = det[0]
        lang = det[2][0][1]
        perc = det[2][0][2]
        return is_reliable, lang, perc
    except Exception as e:
        return "pycld2_error", "pycld2_error", "pycld2_error"

tqdm.pandas(desc="Detecting languages")
cultrax[['is_reliable', 'lang', 'perc']] = cultrax['text'].progress_apply(detect_language).apply(pd.Series)

Detecting languages: 100%|██████████| 5208062/5208062 [12:59<00:00, 6681.97it/s] 


In [11]:
def remove_invalid_utf8(text):
    pattern = re.compile(r'(?![ƏəİIıÖöÜüĞğÇçŞş])([^\x00-\x7F]+)')
    cleaned_text = pattern.sub('', text)
    return cleaned_text

utf_errors = cultrax[cultrax['lang'] == 'pycld2_error']
cultrax = cultrax[cultrax['lang'] != 'pycld2_error']
utf_errors['text'] = utf_errors['text'].apply(remove_invalid_utf8)                          
utf_errors.drop(columns=['is_reliable', 'lang', 'perc'], axis=1, inplace=True)
utf_errors[['is_reliable', 'lang', 'perc']] = utf_errors['text'].apply(detect_language).apply(pd.Series)
cultrax = pd.concat(objs=[cultrax, utf_errors], axis=0)
cultrax = cultrax[cultrax['lang'] == "az"]

In [14]:
cultrax

,text,url,source,domain,is_reliable,lang,perc
0,Mühacirət mövzusunun öyrənilməsi ictimai-siyas...,https://azertag.az/xeber/Muhaciret_movzusunun_...,mC4,azertag,True,az,99
1,Raylar Antaliyada Falez Qovşağına Çıxdı Üçüncü...,https://az.rayhaber.com/2020/10/antalya-3-etap...,mC4,rayhaber,True,az,99
2,Avropaya qaz ixracı birgə tranzit sazişlərinin...,https://az.trend.az/business/energy/1334192.html,mC4,trend,True,az,99
3,Palitra qəzetiSərnişin avtobuslarını çirkləndi...,http://www.palitranews.az/news.php?id=65577,mC4,palitranews,True,az,99
4,Oxunub: 912 10:37 19 İyun 2021\nMərkəzdən Ordu...,https://ordu.az/ru/news/192570/respondentlerin...,mC4,ordu,True,az,99
...,...,...,...,...,...,...,...
132689,Azərbaycanın kosmik proqramı Azərbaycan hökum...,https://az.wikipedia.org/wiki/Az%C9%99rbaycan%...,wiki,wikipedia,True,az,99
134613,Alber Mari Lebur (; ) fransız rəssam-peyzajist...,https://az.wikipedia.org/wiki/Alber%20Mari%20L...,wiki,wikipedia,True,az,99
161691,Şəki Xalq Tətbiqi Sənəti Muzeyi Şəki Xalq Tətb...,https://az.wikipedia.org/wiki/%C5%9E%C9%99ki%2...,wiki,wikipedia,True,az,99
171555,Balta-Limanı ticarət konvensiyası Osmanlı imp...,https://az.wikipedia.org/wiki/Balta-Liman%20ko...,wiki,wikipedia,True,az,82


In [30]:
print(cultrax.sample(1).iloc[0]['text'])

BakuTime.com – Porsche 911 fandomunda “qızardılmış yumurta” (fried-egg) fənərləri, ümumiyyətlə qeyri-adi görünüşü və Porsche üçün ilk olaraq su ilə soyudulmuş düz-6 (water-cooled flat-6) mühərriki ilə 996-dan daha qütbləşən nəsil yoxdur. O, digər nəsillərin etdiyi sevgini tez-tez almır, lakin Porsche-nin ən son birdəfəlik yaradıcılığını nümayiş etdirməsi ilə bu, dəyişə bilər.
Porsche bu həftə avtomobil istehsalçısının Porsche Club of America üçün hazırlanmış tək konsept olan 911 Classic Club Coupe-ni təqdim etdi. Donor avtomobili 1998-ci ildə yığılıb və Virciniya ştatındakı dilerlərin birində sönük vəziyyətdə aşkar edilib. Avtomobil Almaniyaya qaytarıldı, burada tamamilə fərqli bir şeyə çevrilmək üçün ilk addımda tamamilə söküldü.
BəyənilmişPostlar
Renault Megane 27 ildən sonra istehsalını dayandırdı
Avtomobil almaq istəyənlərin NƏZƏRİNƏ: Zəncir, yoxsa qayış mühərrikli maşın yaxşı yol yoldaşıdır? – FOTO
Xiaomi elektrik avtomobili barədə nə bilirsiz?
996-nın gövdəsi ilk dəfə 996-cı nəsi